# Scalar Multiplication

**Module 06** | 06-elliptic-curves

*Double-and-add, n*P*

## Objectives

By the end of this notebook you will be able to:

1. Understand the core ideas behind **scalar multiplication**.
2. Explore these concepts interactively using SageMath.
3. Build intuition through hands-on computation and visualization.

## Prerequisites

- Completion of [Group Structure and Order](06d-group-structure-and-order.ipynb).
- Concepts and notation introduced in the previous notebook.

## Repeated Addition

In [ ]:
# n*P means P + P + ... + P (n times)
E = EllipticCurve(GF(101), [1, 1])
P = E.random_point()
print(f'P = {P}')
print(f'5*P = {5*P}')

## Double-and-Add Algorithm

In [ ]:
# Efficient scalar multiplication using binary expansion of n
def double_and_add(P, n):
    R = P.curve()(0)  # point at infinity
    Q = P
    while n > 0:
        if n % 2 == 1:
            R = R + Q
        Q = Q + Q
        n = n // 2
    return R
# TODO: test against SageMath's built-in n*P

## The ECDLP

In [ ]:
# Given P and Q = n*P, finding n is the ECDLP
E = EllipticCurve(GF(101), [1, 1])
P = E.random_point()
n_secret = randint(1, P.order()-1)
Q = n_secret * P
print(f'P = {P}, Q = {Q}')
print(f'Can you find n such that Q = n*P?')

## Exercises

Try these on your own before moving on:

1. **Exercise 1:** *(TODO: add exercise)*
2. **Exercise 2:** *(TODO: add exercise)*
3. **Exercise 3:** *(TODO: add exercise)*

## Summary

In this notebook we explored **scalar multiplication**.  Key takeaways:

- *(TODO: summarize key point 1)*
- *(TODO: summarize key point 2)*
- *(TODO: summarize key point 3)*

**Next:** [ECDH and ECDSA](06f-ecdh-and-ecdsa.ipynb)